In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import statsmodels.api as sm
import datetime as dt
import seaborn as sns
import scipy.stats as st

### Table of Contents

* [I.Téléchargement des données](#chapter1)
    * [1) Clients de la base de donnée - Sexe et date de naissance](#section_1_1)
    * [2) Produits de la base de donnée - Prix et catégorie](#section_1_2)
    * [3) Transactions enregristrées](#section_1_3)
* [II. Nettoyage de données](#chapter2)
    * [1) Valeurs abbérantes](#section_2_1)
        * [a. Suppression des articles dont le prix est <= 0](#section_2_1_1)
        * [b. Suppression des transactions TEST](#section_2_1_2)
    * [2) Valeurs manquantes](#section_2_2)
        * [a. Jointure des dataframes ](#section_2_2_1)
        * [b. Identification des NaN](#section_2_2_2)
        * [c. Combler les valeurs manquantes](#section_2_2_3)
* [III. Mise en forme pour classement des données selon mois, jour, date ](#chapter3)

## I.Téléchargement des données <a class="anchor" id="chapter1"></a>

### 1) Clients de la base de donnée - Sexe et date de naissance <a class="anchor" id="section_1_1"></a>

In [2]:
customers = pd.read_csv("customers.csv")
customers

,client_id,sex,birth
0,c_4410,f,1967
1,c_7839,f,1975
2,c_1699,f,1984
3,c_5961,f,1962
4,c_5320,m,1943
...,...,...,...
8618,c_7920,m,1956
8619,c_7403,f,1970
8620,c_5119,m,1974
8621,c_5643,f,1968


In [3]:
customers.describe(include='all')

,client_id,sex,birth
count,8623,8623,8623.000000
unique,8623,2,NaN
top,c_7917,f,NaN
freq,1,4491,NaN
mean,NaN,NaN,1978.280877
std,NaN,NaN,16.919535
min,NaN,NaN,1929.000000
25%,NaN,NaN,1966.000000
50%,NaN,NaN,1979.000000
75%,NaN,NaN,1992.000000


### 2) Produits de la base de donnée - Prix et catégorie<a class="anchor" id="section_1_2"></a>

In [4]:
products = pd.read_csv("products.csv")
products

,id_prod,price,categ
0,0_1421,19.99,0
1,0_1368,5.13,0
2,0_731,17.99,0
3,1_587,4.99,1
4,0_1507,3.99,0
...,...,...,...
3282,2_23,115.99,2
3283,0_146,17.14,0
3284,0_802,11.22,0
3285,1_140,38.56,1


In [5]:
products.describe(include='all')

,id_prod,price,categ
count,3287,3287.000000,3287.000000
unique,3287,NaN,NaN
top,0_174,NaN,NaN
freq,1,NaN,NaN
mean,NaN,21.856641,0.370246
std,NaN,29.847908,0.615387
min,NaN,-1.000000,0.000000
25%,NaN,6.990000,0.000000
50%,NaN,13.060000,0.000000
75%,NaN,22.990000,1.000000


### 3) Transactions enregristrées<a class="anchor" id="section_1_3"></a>

In [6]:
transactions = pd.read_csv("transactions.csv", parse_dates=[1])
transactions

,id_prod,date,session_id,client_id
0,0_1483,2021-04-10 18:37:28.723910,s_18746,c_4450
1,2_226,2022-02-03 01:55:53.276402,s_159142,c_277
2,1_374,2021-09-23 15:13:46.938559,s_94290,c_4270
3,0_2186,2021-10-17 03:27:18.783634,s_105936,c_4597
4,0_1351,2021-07-17 20:34:25.800563,s_63642,c_1242
...,...,...,...,...
337011,1_671,2021-05-28 12:35:46.214839,s_40720,c_3454
337012,0_759,2021-06-19 00:19:23.917703,s_50568,c_6268
337013,0_1256,2021-03-16 17:31:59.442007,s_7219,c_4137
337014,2_227,2021-10-30 16:50:15.997750,s_112349,c_5


In [7]:
transactions.describe(include='all')

,id_prod,date,session_id,client_id
count,337016,337016,337016,337016
unique,3266,336855,169195,8602
top,1_369,test_2021-03-01 02:30:02.237413,s_0,c_1609
freq,1081,13,200,12855


## II. Nettoyage de données<a class="anchor" id="chapter2"></a>

### 1) Valeurs abbérantes<a class="anchor" id="section_2_1"></a>

#### a. Suppression des articles dont le prix est <= 0<a class="anchor" id="section_2_1_1"></a>

In [8]:
products = products.sort_values(by = 'price', ascending = False)
products = products[(products['price'] > 0) | (products['price'].isnull())]
products

,id_prod,price,categ
947,2_2,300.00,2
724,2_76,254.44,2
394,2_158,247.22,2
1436,2_167,236.99,2
2779,2_30,233.54,2
...,...,...,...
1530,0_1620,0.80,0
1211,0_1844,0.77,0
370,0_120,0.66,0
2272,0_528,0.62,0


#### b. Suppression des transactions TEST<a class="anchor" id="section_2_1_2"></a>

3 possibilités :
- En ajoutant un masque sur les date dont la longueur serait suppérieur à la date classique. Les données Test contienent la mention "test" au début de la date
- En supprimant les transaction des client_id "ct_" qui sont les users testeurs 
- En supprimant les transaction des id_prod "T_" qui sont les produits testeurs 

In [9]:
transactions['date'] = transactions['date'].astype('str')
transactions = transactions[transactions['client_id'].str.match('c_')]
mask = (transactions['date'].str.len() <= len('2021-04-10 18:37:28.723910')) 
transactions = transactions.loc[mask]
transactions = transactions.loc[~transactions["session_id"].isin(['s_0'])]
transactions

,id_prod,date,session_id,client_id
0,0_1483,2021-04-10 18:37:28.723910,s_18746,c_4450
1,2_226,2022-02-03 01:55:53.276402,s_159142,c_277
2,1_374,2021-09-23 15:13:46.938559,s_94290,c_4270
3,0_2186,2021-10-17 03:27:18.783634,s_105936,c_4597
4,0_1351,2021-07-17 20:34:25.800563,s_63642,c_1242
...,...,...,...,...
337011,1_671,2021-05-28 12:35:46.214839,s_40720,c_3454
337012,0_759,2021-06-19 00:19:23.917703,s_50568,c_6268
337013,0_1256,2021-03-16 17:31:59.442007,s_7219,c_4137
337014,2_227,2021-10-30 16:50:15.997750,s_112349,c_5


### 2) Valeurs manquantes<a class="anchor" id="section_2_2"></a>

#### a. Jointure des dataframes <a class="anchor" id="section_2_2_1"></a>

In [10]:
info_id = pd.merge(customers, transactions, on='client_id', how='right')
info_id = pd.merge(products, info_id, on='id_prod', how='right')
info_id

,id_prod,price,categ,client_id,sex,birth,date,session_id
0,2_2,300.0,2.0,c_4958,m,1999,2021-03-26 04:33:19.697459,s_11582
1,2_2,300.0,2.0,c_4958,m,1999,2021-12-07 06:52:26.810576,s_130658
2,2_2,300.0,2.0,c_4958,m,1999,2021-12-04 15:45:14.708313,s_129351
3,2_2,300.0,2.0,c_2329,f,1996,2021-05-02 18:34:47.753888,s_28956
4,2_2,300.0,2.0,c_5237,m,1999,2021-06-12 08:44:00.882917,s_47558
...,...,...,...,...,...,...,...,...
336811,0_2245,NaN,NaN,c_8138,f,1984,2021-10-04 21:20:27.540982,s_100047
336812,0_2245,NaN,NaN,c_7416,m,1933,2021-11-12 09:25:22.905886,s_118458
336813,0_2245,NaN,NaN,c_4283,f,1976,2021-09-01 02:46:38.931757,s_83377
336814,0_2245,NaN,NaN,c_6236,f,1976,2022-02-25 00:08:08.736068,s_170426


#### b. Identification des NaN<a class="anchor" id="section_2_2_2"></a>

In [11]:
NaN_price = info_id[info_id['price'].isna()]
NaN_price = NaN_price.sort_values(by = 'id_prod', ascending = False)
NaN_price

,id_prod,price,categ,client_id,sex,birth,date,session_id
336713,0_2245,NaN,NaN,c_4505,m,1976,2022-01-09 09:23:31.000720,s_147220
336778,0_2245,NaN,NaN,c_4973,f,1983,2021-05-06 20:14:22.522372,s_30824
336788,0_2245,NaN,NaN,c_553,f,1949,2021-12-23 05:26:36.260066,s_138599
336787,0_2245,NaN,NaN,c_1514,f,1982,2021-10-04 13:36:19.612226,s_99885
336786,0_2245,NaN,NaN,c_8096,f,1989,2021-06-09 10:24:58.224573,s_46210
...,...,...,...,...,...,...,...,...
336744,0_2245,NaN,NaN,c_8524,f,1982,2021-11-20 20:21:06.505658,s_122593
336743,0_2245,NaN,NaN,c_8240,f,1978,2021-04-10 09:22:57.768041,s_18566
336742,0_2245,NaN,NaN,c_1551,m,1972,2021-06-17 20:34:06.230094,s_50022
336741,0_2245,NaN,NaN,c_8286,f,1980,2021-10-26 03:31:42.296144,s_110231


#### c. Combler les valeurs manquantes<a class="anchor" id="section_2_2_3"></a>

On peut identifier ici que la catégorie 0 a posé problème pour certaines transactions. On remplace donc la catégorie NaN par 0 et le prix par la moyenne des prix de cette catégorie.

In [12]:
m0 = products[products.categ == 0]['price'].mean()
info_id['categ'] = info_id['categ'].fillna(value = 0)
info_id['price'] = info_id['price'].fillna(value = m0)
info_id

,id_prod,price,categ,client_id,sex,birth,date,session_id
0,2_2,300.000000,2.0,c_4958,m,1999,2021-03-26 04:33:19.697459,s_11582
1,2_2,300.000000,2.0,c_4958,m,1999,2021-12-07 06:52:26.810576,s_130658
2,2_2,300.000000,2.0,c_4958,m,1999,2021-12-04 15:45:14.708313,s_129351
3,2_2,300.000000,2.0,c_2329,f,1996,2021-05-02 18:34:47.753888,s_28956
4,2_2,300.000000,2.0,c_5237,m,1999,2021-06-12 08:44:00.882917,s_47558
...,...,...,...,...,...,...,...,...
336811,0_2245,11.732795,0.0,c_8138,f,1984,2021-10-04 21:20:27.540982,s_100047
336812,0_2245,11.732795,0.0,c_7416,m,1933,2021-11-12 09:25:22.905886,s_118458
336813,0_2245,11.732795,0.0,c_4283,f,1976,2021-09-01 02:46:38.931757,s_83377
336814,0_2245,11.732795,0.0,c_6236,f,1976,2022-02-25 00:08:08.736068,s_170426


## III. Mise en forme pour classement des données selon mois, jour, date <a class="anchor" id="chapter3"></a>

In [13]:
info_id['date_frmt'] = info_id['date']
info_id.loc[:,['categ']] = info_id['id_prod'].astype(str).str[0]
info_id['age'] = info_id['date'].astype(str).str[0:4]
info_id.loc[:,['birth']] = info_id['birth'].astype(str)
info_id.loc[:,['age']] = info_id['age'].astype(float)
info_id.loc[:,['birth']] = info_id['birth'].astype(float)
info_id.loc[:,['age']] = info_id['age'] - info_id['birth']
info_id['hour'] = info_id['date'].str[11:16]
info_id.loc[:,['date']] = info_id['date'].str[0:10]
info_id.loc[:,['date']] = pd.to_datetime(info_id['date'])
info_id['month'] = info_id['date'].dt.strftime('%B')
info_id['jour'] = info_id['date'].dt.strftime('%A')
info_id['day'] = info_id['date'].dt.strftime('%d')
info_id['greg'] = info_id['date'].map(dt.datetime.toordinal)
info_id = info_id.sort_values(by = 'greg', ascending = True)
info_id

,id_prod,price,categ,client_id,sex,birth,date,session_id,date_frmt,age,hour,month,jour,day,greg
242371,0_1870,9.06,0,c_5196,m,1976.0,2021-03-01,s_393,2021-03-01 19:18:06.898158,45.0,19:18,March,Monday,01,737850
241596,0_1338,9.28,0,c_6990,f,1986.0,2021-03-01,s_319,2021-03-01 15:14:28.717079,35.0,15:14,March,Monday,01,737850
333143,0_1142,3.42,0,c_6983,m,1989.0,2021-03-01,s_137,2021-03-01 06:13:35.623382,32.0,06:13,March,Monday,01,737850
313870,0_560,4.53,0,c_4103,f,1980.0,2021-03-01,s_233,2021-03-01 11:25:45.668969,41.0,11:25,March,Monday,01,737850
261129,0_1029,7.99,0,c_7246,m,1976.0,2021-03-01,s_116,2021-03-01 05:26:21.487685,45.0,05:26,March,Monday,01,737850
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46883,1_296,24.67,1,c_1092,f,1972.0,2022-02-28,s_171938,2022-02-28 00:32:05.527835,50.0,00:32,February,Monday,28,738214
169393,1_448,13.79,1,c_359,f,1988.0,2022-02-28,s_172385,2022-02-28 22:16:21.571957,34.0,22:16,February,Monday,28,738214
119933,1_388,16.70,1,c_6878,m,1950.0,2022-02-28,s_172405,2022-02-28 22:52:38.395706,72.0,22:52,February,Monday,28,738214
298516,0_1400,5.15,0,c_8294,f,1966.0,2022-02-28,s_172037,2022-02-28 05:23:22.894171,56.0,05:23,February,Monday,28,738214


In [14]:
info_id.to_csv('info_id.csv', index = False)